# BPR Bayesian Personalized Ranking

---

ThetaLog.com

In [1]:
# Load các thư viện cần thiết
import os
import math
import pathlib
import numpy as np
import pandas as pd
from time import time
from urllib.request import urlopen
from zipfile import ZipFile
from scipy.sparse import csr_matrix, dok_matrix
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

np.random.seed(12)

In [2]:
test_path = './MINDsmall_dev/behaviors.tsv'
train_path = './MINDsmall_train/behaviors.tsv'

output_test_path = './datafile/small_dev.csv'
output_train_path = './datafile/small_train.csv'

result_test_file = 'small_dev'
result_train_file = 'small_train'

user_min = 10
item_min = 10

test_ratio = 0.5

In [3]:
def load_behaviors(path):
    df = pd.read_csv(
        path,
        sep = "\t",
        names = ["id", "user_id", "time", "history", "impressions"]
    )
    return df

def write_file(df, path):
    t_start = time()
    print("Processing", path)
    with open(path, "w") as file:
        for i, row in df.iterrows():
            for new_id in str(row.history).split(' '):
                if row.user_id[1:].isnumeric() and new_id[1:].isnumeric():
                    file.write(row.user_id[1:] + "," + new_id[1:] + "\n")

    t_end = time()
    print("Processed in: {:.2f} Seconds".format(t_end - t_start))

def write_data_file(test_path, train_path, output_test_path, output_train_path):
    test_df, train_df = load_behaviors(test_path), load_behaviors(train_path)

    write_file(test_df, output_test_path)
    write_file(train_df, output_train_path)

In [4]:
write_data_file(test_path, train_path, output_test_path, output_train_path)

Processing ./datafile/small_dev.csv
Processed in: 13.33 Seconds
Processing ./datafile/small_train.csv
Processed in: 28.04 Seconds


In [5]:
def load_file(path):
    df = pd.read_csv(
        path,
        header=None,
        names=['user_id', 'item_id']
    )
    df.reindex(columns = ['user_id', 'item_id'])
    return df.reset_index(drop = True)

def load_data(test_path, train_path):
    test_df, train_df = load_file(test_path), load_file(train_path)
    
    return test_df, train_df

In [6]:
test_df, train_df = load_data(output_test_path, output_train_path)

In [7]:
def filter_df(df, user_min, item_min):
    if df is None:
        return

    t_start = time()

    user_counts = df.groupby("user_id").size()
    user_subset = np.in1d(
        df.user_id, user_counts[user_counts >= item_min].index
    )

    filter_df = df[user_subset].reset_index(drop=True)

    # find items with 10 or more users
    item_counts = filter_df.groupby("item_id").size()
    item_subset = np.in1d(
        filter_df.item_id, item_counts[item_counts >= user_min].index
    )

    filter_df = filter_df[item_subset].reset_index(drop=True)

    user_counts = filter_df.groupby("user_id").size()
    user_subset = np.in1d(filter_df.user_id, user_counts[user_counts >= item_min].index)

    filter_df = filter_df[user_subset].reset_index(drop=True)

    t_end = time()

    assert (filter_df.groupby("user_id").size() < 5).sum() == 0
    assert (filter_df.groupby("item_id").size() < 5).sum() == 0

    print(filter_df.nunique())
    print(filter_df.shape)
    print("{:.2f} Seconds".format(t_end - t_start))

    return filter_df

def reset_df(df):
    item_enc = LabelEncoder()
    df["item_id"] = item_enc.fit_transform(df["item_id"])

    user_enc = LabelEncoder()
    df["user_id"] = user_enc.fit_transform(df["user_id"])

    assert df.user_id.min() == 0
    assert df.item_id.min() == 0

    return df

In [8]:
filtered_train_df = reset_df(filter_df(train_df, user_min, item_min))
filtered_test_df = reset_df(filter_df(test_df, user_min, item_min))

user_id    34584
item_id    20261
dtype: int64
(4970882, 2)
1.03 Seconds
user_id    33436
item_id    13857
dtype: int64
(2210998, 2)
0.44 Seconds


In [9]:
def convert_to_bpr_mat(dataframe, threshold = 3):
    """
    Chuyển đổi DataFrame MovieLens 100K ban đầu sang ma trận BPR
    Mỗi dòng là Users
    Mỗi cột là Item
    Định dạng ma trận thưa

    :param dataframe: pandas df movielens 100K
    :return bpr_mat: np.array - ma trận thưa bpr
    """
    tempdf = dataframe.copy()
    tempdf['ratings'] = 5
    tempdf['positive'] = tempdf['ratings'].apply(func=lambda x: 0 if x < threshold else 1)

    # Vì tập dữ liệu này đánh index từ 1 nên chuyển sang kiểu category
    # để tránh việc chúng ta có ma trận
    tempdf['user_id'] = tempdf['user_id'].astype('category')
    tempdf['item_id'] = tempdf['item_id'].astype('category')

    bpr_mat = csr_matrix((tempdf['positive'],
                          (tempdf['user_id'].cat.codes,
                           tempdf['item_id'].cat.codes)))
    bpr_mat.eliminate_zeros()
    del tempdf
    return bpr_mat

In [10]:
bpr_train = convert_to_bpr_mat(filtered_train_df)
bpr_test = convert_to_bpr_mat(filtered_test_df)

In [11]:
print(bpr_train.shape)
print(bpr_test.shape)

(34584, 20261)
(33436, 13857)


In [12]:
def split_to_train_test(bpr_mat, test_ratio = 0.2, verbose=True):
    """
    Chia tập dữ liệu ra thành tập train & tập test

    :param bpr_mat: ma trận bpr
    :param test_ratio: float - tỉ lệ test set

    :return train: ma trận bpr train
    :return test: ma trận bpr test
    """
    # Số lượng người dùng
    n_users = bpr_mat.shape[0]
    # Dùng ma trận thưa Dictionary Of Keys tối ưu hơn cho công đoạn này
    train = bpr_mat.copy().todok()
    test = dok_matrix(train.shape) # Lưu ý hiện tại test là ma trận 0

    # với mỗi người dùng u
    # chia số trường hợp nên khuyến nghị với tỉ lệ test_ratio đươc cho
    # phần nào thuộc về test
    for u in range(n_users):
        split_index = bpr_mat[u].indices
        # đếm số trường hợp nên khuyến nghị
        count_positive = split_index.shape[0]
        n_splits = max(min(math.ceil(test_ratio * count_positive), count_positive - 1), 1)
        test_index = np.random.choice(split_index, size=n_splits, replace=False)
        # Xem như dữ liệu chưa biết trong tập train
        train[u, test_index] = 0
        # Xem như dữ liệu nhìn thấy trong tập test
        test[u, test_index] = 1

    train, test = train.tocsr(), test.tocsr()

    # Nếu cần in thông tin ra ngoài
    if verbose:
        print('BPR matrix with %d stored elements' % bpr_mat.nnz)
        print('Train matrix with %d stored elements' % train.nnz)
        print('Test matrix with %d stored elements' % test.nnz)
    return train, test

In [13]:
bpr_train_train, bpr_train_test = split_to_train_test(bpr_train, test_ratio=test_ratio, verbose=True)
bpr_test_train, bpr_test_test = split_to_train_test(bpr_test, test_ratio=test_ratio, verbose=True)

BPR matrix with 832717 stored elements
Train matrix with 407707 stored elements
Test matrix with 425010 stored elements
BPR matrix with 1141694 stored elements
Train matrix with 562564 stored elements
Test matrix with 579130 stored elements


In [14]:
def predict_bpr(W, H, user=None):
    """
    Hàm trả về X_hat

    :param W: ma trận W từ MF
    :param H: ma trận H từ MF
    :param user: người dùng (nếu None mặt định trả về tất cả)

    :return predict_scores: điểm dự đoán từ BPR MF
    """
    if user is None:
        return W @ H.T
    else:
        return W[user] @ H.T

def recommend_bpr(bpr_matrix, predict_score, user, n_rmd_items=None):
    """
    Dự đoán những sản phẩm mà người dùng muốn mua
    Những sản phẩm nào đã thích rồi thì không trả về nữa
    Trả về index theo bpr_matrix (đánh từ 0)

    :param bpr_matrix: ma trận bpr hiện tại
    :param predict_score: điểm dự đoán các item
    :param user: số thứ tự người dùng của predict score
    :param n_rmd_items: số lượng sản phẩm trả về, mặc định tất cả

    :return rmd_items: danh sách các sản phẩm khuyến nghị
    """
    # Số lượng sản phẩm
    n_items = bpr_matrix.shape[1]
    # những sản phẩm đã thích rồi
    liked_items = bpr_matrix[user].indices
    scores = predict_score.copy()

    # index ban đầu khi chưa sắp xếp
    sort_index = np.arange(0, n_items)

    # Xóa các sản phẩm đã mua
    sort_index = np.delete(sort_index, liked_items)
    scores = np.delete(scores, liked_items)

    # sắp xếp và trả về theo số thứ tự của score
    arg_sort = np.argsort(-scores)

    # dùng sort_index để lấy số thứ tự ban đầu
    rmd_items = sort_index[arg_sort]

    if len(rmd_items) >= n_rmd_items and n_rmd_items is not None:
        rmd_items = rmd_items[: n_rmd_items]
    return rmd_items

def auc_score(predict_mat, bpr_mat):
    """
    Tính Area under the ROC curve (AUC)
    cho bài toán hệ khuyến nghị

    :param predict_mat: ma trận dữ đoán bpr mf
    :param bpr_mat: ma trận train hoặc test
    :return auc: area under the roc curve
    """
    auc = 0.0
    n_users, n_items = bpr_mat.shape

    # u và row tương ứng user và bp
    for u in range(n_users):
        y_pred = predict_mat[u]
        y_true = np.zeros(n_items)
        y_true[bpr_mat[u].indices] = 1
        try:
            auc += roc_auc_score(y_true, y_pred)
        except ValueError:
            continue
    auc /= n_users
    return auc

In [15]:
def learn_bpr_mf_sgd(bpr_mat, pos, neg, W = None, H = None, alpha=0.01, lamb=0.01, k=12, n_iters=10000):
    """
    Thuật toán học BPR MF SGD (một điểm dữ liệu duy nhất)

    :param bpr_mat: ma trận bpr
    :param alpha: hệ số learning rate
    :param lamb: hệ số lambda của bình thường hóa regularization
    :param k: số lượng latent factor trong bài toán MF
    :param n_iters: số vòng lặp

    :return W: ma trận W
    :return H: ma trận H
    """
    n_users, n_items = bpr_mat.shape
    # Khởi tạo ma trận W và ma trận H
    if W is None:
        W = np.ones(shape=(n_users, k))
    if H is None:
        H = np.ones(shape=(n_items, k))

    # lặp
    for _ in range(n_iters):
        # ngẫu nghiên 3 bộ (u,i,j) từ D_S
        u = np.random.randint(0, n_users)
        if len(pos[u]) == 0:
            continue
        i = pos[u][np.random.randint(0, len(pos[u]))]
        j = neg[u][np.random.randint(0, len(neg[u]))]

        # Tính xuij
        xui = (W[u] * H[i]).sum()
        xuj = (W[u] * H[j]).sum()
        xuij = xui - xuj

        # mũ tự nhiên e của xuij
        exp_xuij = np.exp(xuij)

        # sgd cho tham số Theta (W và H)
        W[u] = W[u] + alpha * ( exp_xuij / (1+exp_xuij) * (H[i] - H[j]) + lamb * W[u])
        H[i] = H[i] + alpha * ( exp_xuij / (1+exp_xuij) * W[u] + lamb * H[i])
        H[j] = H[j] + alpha * ( exp_xuij / (1+exp_xuij) * (-W[u]) + lamb * H[j])
    return W, H

In [16]:
W_train, H_train = None, None

# Tập các sản phẩm nên khuyến nghị
pos = np.split(bpr_train_train.indices, bpr_train_train.indptr)[1:-1]
# Tập các sản phẩm không nên khuyến nghị
neg = [np.setdiff1d(np.arange(0, bpr_train_train.shape[1], 1), e) for e in pos]

In [17]:
W_train, H_train = learn_bpr_mf_sgd(
    bpr_train_train,
    pos,
    neg,
    W = W_train,
    H = H_train,
    alpha=0.001,
    lamb=0.02,
    k=60,
    n_iters=50000
)

pred_train = predict_bpr(W_train, H_train)

train_train = auc_score(pred_train, bpr_train_train)
train_test = auc_score(pred_train, bpr_train_test)
print('Train-Train: %f' % train_train)
print('Train-Test: %f' % train_test)

Train-Train: 0.855999
Train-Test: 0.848096


In [18]:
output_folder = os.path.join('./result', datetime.now().strftime("%d-%m-%Y %H-%M"))

pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)

np.save(os.path.join(output_folder, result_train_file + '_W.npy'), W_train)
np.save(os.path.join(output_folder, result_train_file + '_H.npy'), H_train)

with open(os.path.join(output_folder, result_train_file + '.txt'), 'w') as file:
    file.write('Train-Train: %f' % train_train + "\n")
    file.write('Train-Test: %f' % train_test + "\n")

In [19]:
W_test, H_test = None, None
# Tập các sản phẩm nên khuyến nghị
pos = np.split(bpr_test_train.indices, bpr_test_train.indptr)[1:-1]
# Tập các sản phẩm không nên khuyến nghị
neg = [np.setdiff1d(np.arange(0, bpr_test_train.shape[1], 1), e) for e in pos]

In [20]:
W_test, H_test = learn_bpr_mf_sgd(
    bpr_test_train,
    pos,
    neg,
    W = W_test,
    H = H_test,
    alpha=0.001,
    lamb=0.02,
    k=60,
    n_iters=50000
)

pred_test = predict_bpr(W_test, H_test)

test_train = auc_score(pred_test, bpr_test_train)
test_test = auc_score(pred_test, bpr_test_test)
print('Test-Train: %f' % test_train)
print('Test-Test: %f' % test_test)

Test-Train: 0.823057
Test-Test: 0.818639


In [21]:
output_folder = os.path.join('./result', datetime.now().strftime("%d-%m-%Y %H-%M"))

pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)

np.save(os.path.join(output_folder, result_test_file + '_W.npy'), W_test)
np.save(os.path.join(output_folder, result_test_file + '_H.npy'), H_test)

with open(os.path.join(output_folder, result_test_file + '.txt'), 'w') as file:
    file.write('Test-Train: %f' % test_train + "\n")
    file.write('Test-Test: %f' % test_test + "\n")

In [22]:
u = 300
n_rmd_items = 5
score = predict_bpr(W_test, H_test, u)
rmd_items = recommend_bpr(bpr_test_train, score, u, n_rmd_items)
print(rmd_items)

[  55 9095 9747 9190 3544]


# Tham khảo

01. Steffen Rendle, Christoph Freudenthaler, Zeno Gantner and Lars Schmidt-Thieme. BPR: Bayesian Personalized Ranking from Implicit Feedback. 
02. Weike Panand, Li Chen. GBPR: Group Preference Based Bayesian Personalized Ranking for One-Class Collaborative Filtering. Proceedings of the Twenty-Third International Joint Conference on Artificial Intelligence. https://www.aaai.org/ocs/index.php/IJCAI/IJCAI13/paper/viewFile/6316/7124
03. Michael D. Ekstrand, Joseph A Konstan. Personalized Ranking (with Daniel Kluver). Matrix Factorization and Advanced Techniques - University of Minnesota. https://www.coursera.org/lecture/matrix-factorization/personalized-ranking-with-daniel-kluver-s3XJo
04. Kim Falk. Practical Recommender Systems. Manning Publications.
05. Ethen (MingYu) Liu. Bayesian Personalized Ranking. http://ethen8181.github.io/machine-learning/recsys/4_bpr.html
06. Alfredo Láinez Rodrigo, Luke de Oliveira. Distributed Bayesian Personalized Ranking in Spark. https://stanford.edu/~rezab/classes/cme323/S16/projects_reports/rodrigo_oliveira.pdf